In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import dask.dataframe as dd
import re

Regular pandas wasn't working. Learned about dask and it worked beautifully. The below is only needed the first time. Once you save the filtered csv you can skip the steps to filter it down to just 2018 onwards.

In [ ]:
seattle_df = dd.read_csv('..\data\checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [ ]:
filtered_seattle_df = seattle_df[seattle_df['CheckoutYear'] >= 2018]

In [ ]:
filtered_seattle_df.head()

In [ ]:
result = filtered_seattle_df.compute()

In [ ]:
result.to_csv('..\\data\\filtered_checkouts_by_title.csv', index=False)

Steps to load and run from the filtered data that I saved to a CSV file.

In [111]:
seattle_df = dd.read_csv('..\data\\filtered_checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [112]:
seattle_df.head()

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear
0,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020
1,Digital,OverDrive,AUDIOBOOK,2022,9,2,"The Free Bastards: The Lot Lands Series, Book ...",9780593414682,Jonathan French,"Fantasy, Fiction",Books on Tape,2021
2,Digital,OverDrive,EBOOK,2022,9,8,"Dawn: Warriors: The New Prophecy Series, Book 3",9780061757433,Erin Hunter,"Fantasy, Juvenile Fiction, Juvenile Literature",HarperCollins Publishers Inc.,2007
3,Physical,Horizon,BOOK,2022,9,1,Black box : a record of the catastrophe. Volum...,"162963123X, 9781629631233",NaN,"Social change, Social change Literary collections","PM Press,",[2015]
4,Digital,OverDrive,AUDIOBOOK,2023,3,2,Blood Games (unabridged),9781452698052,Chloe Neill,"Fantasy, Fiction, Literature, Romance","Tantor Media, Inc",2015


In [113]:
seattle_df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 12 entries, UsageClass to PublicationYear
dtypes: object(9), int64(3)

In [114]:
result = seattle_df.compute()

In [115]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12415765 entries, 0 to 248235
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UsageClass       object
 1   CheckoutType     object
 2   MaterialType     object
 3   CheckoutYear     int64 
 4   CheckoutMonth    int64 
 5   Checkouts        int64 
 6   Title            object
 7   ISBN             object
 8   Creator          object
 9   Subjects         object
 10  Publisher        object
 11  PublicationYear  object
dtypes: int64(3), object(9)
memory usage: 1.2+ GB


Trying to clean ISBN and split into ISBN-10 and ISBN-13. Changing to one by one so Dask doesn't throw an error. Dask did not work. ):

Finding the corresponding lengths to sort into the correct columns.

Trying for loop and chunk iterator.

In [116]:
def process_chunk(chunk):
    chunk_copy = chunk.copy()
    chunk_copy['ISBN'] = chunk_copy['ISBN'].astype(str)
    split_isbn = chunk_copy['ISBN'].str.split(',', expand=True)
    chunk_copy.loc[:, 'Possible ISBN-10'] = split_isbn[0].str.strip()
    chunk_copy.loc[:, 'Possible ISBN-13'] = None
    if split_isbn.shape[1] > 1:
        chunk_copy.loc[:, 'Possible ISBN-13'] = split_isbn[1].str.strip()
    chunk_copy.loc[:, 'ISBN-10'] = chunk_copy['Possible ISBN-10'].where(chunk_copy['Possible ISBN-10'].str.len() == 10, np.nan)
    chunk_copy.loc[:, 'ISBN-13'] = chunk_copy['Possible ISBN-13'].where(chunk_copy['Possible ISBN-13'].str.len() == 13, chunk_copy['Possible ISBN-10'])
    chunk_copy = chunk_copy.drop(columns=['Possible ISBN-10', 'Possible ISBN-13'])
    return chunk_copy

In [117]:
chunk_size = 10**5
chunks = [process_chunk(result[i:i+chunk_size]) for i in range(0, result.shape[0], chunk_size)]

In [118]:
result_processed = pd.concat(chunks, ignore_index=True)

In [119]:
result_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12415765 entries, 0 to 12415764
Data columns (total 14 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UsageClass       object
 1   CheckoutType     object
 2   MaterialType     object
 3   CheckoutYear     int64 
 4   CheckoutMonth    int64 
 5   Checkouts        int64 
 6   Title            object
 7   ISBN             object
 8   Creator          object
 9   Subjects         object
 10  Publisher        object
 11  PublicationYear  object
 12  ISBN-10          object
 13  ISBN-13          object
dtypes: int64(3), object(11)
memory usage: 1.3+ GB


In [120]:
result_processed.head()

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13
0,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020,NaN,9781781100769
1,Digital,OverDrive,AUDIOBOOK,2022,9,2,"The Free Bastards: The Lot Lands Series, Book ...",9780593414682,Jonathan French,"Fantasy, Fiction",Books on Tape,2021,NaN,9780593414682
2,Digital,OverDrive,EBOOK,2022,9,8,"Dawn: Warriors: The New Prophecy Series, Book 3",9780061757433,Erin Hunter,"Fantasy, Juvenile Fiction, Juvenile Literature",HarperCollins Publishers Inc.,2007,NaN,9780061757433
3,Physical,Horizon,BOOK,2022,9,1,Black box : a record of the catastrophe. Volum...,"162963123X, 9781629631233",NaN,"Social change, Social change Literary collections","PM Press,",[2015],162963123X,9781629631233
4,Digital,OverDrive,AUDIOBOOK,2023,3,2,Blood Games (unabridged),9781452698052,Chloe Neill,"Fantasy, Fiction, Literature, Romance","Tantor Media, Inc",2015,NaN,9781452698052


Checking to see if either of the ISBN columns have complete isbn data so I can possibly drop the other isbn columns.

In [66]:
result_processed['ISBN-13'].isna().sum()

0

In [67]:
result_processed['ISBN-10'].isna().sum()

11397338

In [71]:
result_processed['PublicationYear'].isna().sum()

106226

ISBN-13 has a record for every book so I am dropping the ISBN and the ISBN-10 columns. **Not doing this anymore because this was a LIE lol.**

In [68]:
result_processed = result_processed.drop(columns=['ISBN', 'ISBN-10'])

In [121]:
result_processed.tail(5)

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13
12415760,Digital,OverDrive,EBOOK,2023,3,3,Bo the Brave,9781338323443,Rebecca Elliott,"Juvenile Fiction, Juvenile Literature","Scholastic, Inc. (US)",2020,NaN,9781338323443
12415761,Digital,OverDrive,AUDIOBOOK,2023,3,3,The Lido (unabridged),9781508255796,Libby Page,"Fiction, Literature",Simon & Schuster - Audiobooks,2018,NaN,9781508255796
12415762,Digital,OverDrive,EBOOK,2022,9,2,"The Catch: US Marshals Series, Book 3",9781493434176,Lisa Harris,"Fiction, Romance, Suspense, Christian Fiction",Baker Publishing Group,2022,NaN,9781493434176
12415763,Digital,OverDrive,EBOOK,2022,9,3,Escape from the Nether!,9780593430699,Nick Eliopulos,"Juvenile Fiction, Juvenile Literature","Random House, Inc.",2022,NaN,9780593430699
12415764,Physical,Horizon,BOOK,2022,9,6,Fancy Nancy and the wedding of the century / w...,"0062083198, 0062083201, 9780062083197, 9780062...","O'Connor, Jane",Fancy Nancy Fictitious character Juvenile fict...,"HarperCollins Publishers,",[2014],0062083198,0062083198


Trying to clean the year column. Decided to keep null values so I can possibly do something with them if I want to.

In [122]:
def clean_year(year):
    if pd.isna(year):
        return year 
    year = str(year) 
    cleaned_year = ''.join(filter(str.isdigit, year))
    return cleaned_year[:4] if len(cleaned_year) >= 4 else None

In [123]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [124]:
result_processed['Cleaned_Year'].unique()

array(['2020', '2021', '2007', '2015', '2017', '2014', '2009', '2022',
       '2011', '2018', '2013', '2008', '2005', '2010', '2016', '2012',
       '2019', '2003', '1998', '1944', '2000', '1992', '2006', nan,
       '2004', '1997', '1999', '1988', '1994', '1986', '2002', '2001',
       '1985', '1987', '1989', '1983', '1990', '1972', '1956', '1970',
       '1995', '1979', '1964', '1996', '1991', '1993', '1955', '1961',
       '1928', '1974', '1976', '1937', '1969', '1980', '1929', '1924',
       '1978', '1966', '1984', '1965', '1941', '1977', '8019', '1982',
       '1957', '1960', '1954', '1953', '1968', '1951', '1939', '8119',
       '1973', '1981', '1971', '1962', '1911', '1940', '1963', None,
       '6219', '1935', '1832', '1921', '1959', '1888', '1926', '1910',
       '1958', '1948', '1975', '1943', '1967', '5199', '1932', '1900',
       '1945', '7719', '1949', '1946', '1950', '7819', '1938', '1952',
       '1927', '1912', '1913', '1931', '1906', '1936', '1934', '2620',
       '188

Discovered that there's some more values that are weird. Trying something else.

In [125]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    year = str(year)
    year = ''.join(filter(lambda x: x.isdigit(), year))
    year = year[-4:]
    return year if len(year) == 4 else np.nan

In [126]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [127]:
result_processed['Cleaned_Year'].unique()

array(['2020', '2021', '2007', '2015', '2017', '2014', '2009', '2022',
       '2011', '2018', '2013', '2008', '2005', '2010', '2016', '2012',
       '2019', '1977', '2003', '1998', '1944', '2000', '1992', '2006',
       nan, '2004', '1997', '1999', '1988', '1994', '1986', '2002',
       '2001', '1987', '1984', '1989', '1983', '1996', '1990', '1972',
       '1985', '1956', '1979', '1995', '1964', '1991', '1993', '1976',
       '1961', '1928', '1974', '1970', '1937', '1965', '1969', '1980',
       '1929', '1924', '1978', '1966', '1975', '6571', '1941', '1982',
       '1957', '1960', '1954', '1953', '1968', '1951', '1939', '1973',
       '1981', '1971', '1962', '1911', '1940', '1963', '1935', '1832',
       '1921', '1959', '1888', '1926', '1910', '1958', '4853', '1943',
       '1967', '1955', '1932', '6484', '1945', '1948', '1949', '1946',
       '1950', '7374', '1938', '1952', '1927', '1912', '1703', '1913',
       '1931', '1900', '1906', '1936', '1934', '1886', '1894', '6669',
       '1

Trying to see what is going on with the weird rows.

In [128]:
result_processed[result_processed['Cleaned_Year'] == '92009']

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13,Cleaned_Year


Adding more cases to clean this further.

In [129]:
def clean_year(year):
    if isinstance(year, str):
        match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
        if match:
            return match.group(1)  
        
        match = re.match(r"(\d{4})(\d{4})", year)
        if match:
            return match.group(1)
        
    return year

In [130]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [131]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

In [190]:
result_processed['Cleaned_Year'].unique()

array([2020., 2021., 2007.,   nan, 2015., 2014., 2009., 2022., 2011.,
       2018., 2013., 2008., 2005., 2010., 2016., 2012., 2019., 2017.,
       1944., 2003., 1992., 2006., 1997., 1999., 1988., 1994., 2002.,
       2000., 1989., 1983., 1972., 1985., 1998., 1995., 2001., 2004.,
       1996., 1993., 1928., 1990., 1991., 1970., 1976., 1937., 1969.,
       1966., 1965., 1961., 1964., 1960., 1954., 1986., 1982., 1968.,
       1951., 1939., 1911., 1974., 1940., 1935., 1921., 1973., 1888.,
       1984., 1926., 1956., 1963., 1910., 1958., 1979., 1981., 1975.,
       1967., 1980., 1977., 1987., 1955., 1932., 1962., 1943., 1948.,
       1959., 1953., 1971., 1957., 1950., 1978., 1952., 1927., 1913.,
       1931., 1900., 1945., 1906., 1936., 1934., 1894., 1924., 1925.,
       1912., 1922., 1903., 1947., 1941., 1938., 1896., 1897., 1905.,
       1901., 1909., 1883., 1873., 1942., 1946., 1895., 1949., 1929.,
       1914., 1916., 1907., 1919., 1874., 1899., 1933., 1898., 1923.,
       1930., 1917.,

In [189]:
result_processed[result_processed['Cleaned_Year'] == 2022.1]

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear,ISBN-10,ISBN-13,Cleaned_Year


Gonna replace some of the easy ones.Run as many times as you need to clean up the above lol.

In [185]:
indexes_to_update = [12303238]

In [186]:
new_year = 2022

In [187]:
result_processed.loc[indexes_to_update, 'PublicationYear'] = new_year

In [188]:
result_processed.loc[indexes_to_update, 'Cleaned_Year'] = new_year

Combining all the cleaning fuctions for next time I run this notebook.

In [191]:
def clean_year(year):
    if pd.isna(year):
        return np.nan
    
    year = str(year)
    match = re.search(r"\[(\d{4})-(\d{2,4})\]", year)
    if match:
        return match.group(1)
    
    match = re.match(r"(\d{4})(\d{4})", year)
    if match:
        return match.group(1)
    
    year = ''.join(filter(lambda x: x.isdigit(), year))
    return year[-4:] if len(year) == 4 else np.nan

In [193]:
result_processed['Cleaned_Year'] = result_processed['PublicationYear'].apply(clean_year)

In [194]:
result_processed['Cleaned_Year'] = pd.to_numeric(result_processed['Cleaned_Year'], errors='coerce')

In [195]:
result_processed['Cleaned_Year'].unique()

array([2020., 2021., 2007., 2015., 2017., 2014., 2009., 2022., 2011.,
       2018., 2013., 2008., 2005., 2010., 2016., 2012., 2019.,   nan,
       2003., 1998., 1944., 2000., 1992., 2006., 2004., 1997., 1999.,
       1988., 1994., 1986., 2002., 2001., 1987., 1989., 1983., 1990.,
       1972., 1985., 1956., 1995., 1979., 1964., 1996., 1991., 1993.,
       1961., 1928., 1974., 1970., 1976., 1937., 1969., 1929., 1924.,
       1978., 1966., 1965., 1941., 1977., 1982., 1957., 1960., 1954.,
       1953., 1968., 1951., 1939., 1973., 1981., 1971., 1962., 1911.,
       1940., 1963., 1980., 1984., 1935., 1832., 1921., 1959., 1888.,
       1926., 1910., 1958., 1975., 1943., 1967., 1955., 1932., 1945.,
       1948., 1949., 1946., 1950., 1938., 1952., 1927., 1912., 1913.,
       1931., 1900., 1906., 1936., 1934., 1886., 1894., 1925., 1922.,
       1899., 1903., 1947., 1896., 1897., 1942., 1917., 1923., 1887.,
       1905., 1901., 1909., 1883., 1893., 1873., 1904., 1891., 1895.,
       1914., 1916.,

Saving to a CSV to not lose all my manual cleaning :)

In [196]:
result_processed.to_csv('cleaned_checkouts_data.csv', index=False)